In [11]:
import numpy as np
import random
import html
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
import rapidfuzz
import tqdm
import matplotlib as plt
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import glob
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

In [12]:
df_employees = pd.read_csv(r"C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\raw\categories_segmentation\Employees\Apollo\dataset_data-apollo_2025-12-12_13-19-06-161.csv")
df_employees.head()

,first_name,last_name,email,mobile_number,personal_email,company_name,company_website,linkedin,full_name,job_title,industry,headline,seniority_level,company_linkedin,functional_level,company_size,city,state,country,company_annual_revenue,company_annual_revenue_clean,company_description,company_total_funding,company_total_funding_clean,keywords,company_technologies,company_linkedin_uid,company_founded_year,company_domain,company_phone,company_street_address,company_full_address,company_state,company_city,company_country,company_postal_code
0,Damir,Klarin,damir.klarin@dahuatech.com,NaN,NaN,Dahua Technology Co. Ltd,http://www.dahuasecurity.com,http://www.linkedin.com/in/damir-klarin-8496283,Damir Klarin,Business Development Manager,Information Technology & Services,NaN,manager,http://www.linkedin.com/company/dahua-technology,sales,23000,NaN,NaN,Croatia,32390000000.00,32.4B,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",NaN,NaN,"dvrs, nvr, ivs, ip cameras, security solution ...","hubspot, react redux, remote, route 53, vuejs",2845844,2001,dahuasecurity.com,+86 571 8768 8883,1199 Bin'an Road,"No.1199, Bin'an Road, Binjiang District, Hangz...",Zhejiang Sheng,Hangzhou Shi,China,NaN
1,Ali,Kloub,NaN,NaN,alikloub@hotmail.com,Dahua Technology Co. Ltd,http://www.dahuasecurity.com,http://www.linkedin.com/in/ali-kloub-b15b2657,Ali Kloub,Regional Sales Director (mena),Information Technology & Services,Regional Director (mena)\nbusiness Growth | St...,director,http://www.linkedin.com/company/dahua-technology,sales,23000,Dubai,Dubai,United Arab Emirates,32390000000.00,32.4B,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",NaN,NaN,"dvrs, nvr, ivs, ip cameras, security solution ...","hubspot, react redux, remote, route 53, vuejs",2845844,2001,dahuasecurity.com,+86 571 8768 8883,1199 Bin'an Road,"No.1199, Bin'an Road, Binjiang District, Hangz...",Zhejiang Sheng,Hangzhou Shi,China,NaN
2,Yahya,Asfari,yahya.asfari@dahuatech.com,NaN,NaN,Dahua Technology Co. Ltd,http://www.dahuasecurity.com,http://www.linkedin.com/in/yahya1asfari,Yahya Asfari,Director Of Business Development | Ai Surveill...,Information Technology & Services,Director Of Business Development | Ai Surveill...,director,http://www.linkedin.com/company/dahua-technology,engineering,23000,Jeddah,Makkah Province,Saudi Arabia,32390000000.00,32.4B,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",NaN,NaN,"dvrs, nvr, ivs, ip cameras, security solution ...","hubspot, react redux, remote, route 53, vuejs",2845844,2001,dahuasecurity.com,+86 571 8768 8883,1199 Bin'an Road,"No.1199, Bin'an Road, Binjiang District, Hangz...",Zhejiang Sheng,Hangzhou Shi,China,NaN
3,Jose,Ruben Trujillo,ruben.trujillo@dahuatech.com,NaN,NaN,Dahua Technology Co. Ltd,http://www.dahuasecurity.com,http://www.linkedin.com/in/jruben-trujillo,Jose Ruben Trujillo,Business Development Director,Information Technology & Services,Business Development Director,director,http://www.linkedin.com/company/dahua-technology,sales,23000,Bogota,Bogota,Colombia,32390000000.00,32.4B,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",NaN,NaN,"dvrs, nvr, ivs, ip cameras, security solution ...","hubspot, react redux, remote, route 53, vuejs",2845844,2001,dahuasecurity.com,+86 571 8768 8883,1199 Bin'an Road,"No.1199, Bin'an Road, Binjiang District, Hangz...",Zhejiang Sheng,Hangzhou Shi,China,NaN
4,Rodolphe,Kassouf,NaN,NaN,rodolphkassouf@gmail.com,Dahua Technology Co. Ltd,http://www.dahuasecurity.com,http://www.linkedin.com/in/rodolphe-kassouf,Rodolphe Kassouf,Director Of Business Development,Information Technology & Services,Director Of Business Development,director,http://www.linkedin.com/company/dahua-technology,sales,23000,Beirut,Beirut Governorate,Lebanon,32390000000.00,32.4B,"Zhejiang Dahua Technology Co., Ltd. (Dahua Tec...",NaN,NaN,"dvrs, nvr, ivs, ip cameras, security solution ...","hubspot, react redux, remote, route 53, vuejs",2845844,2001,dahuasecurity.com,+86 571 8768 8883,1199 Bin'an Road,"No.1199, Bin'an Road, Binjiang Distr

In [13]:
df_employees_c = pd.read_csv(r"C:\Users\LENOVO\Documents\GitHub\Enrichment_database\data\clean\Employees\df_employees_cleaned.csv")
df_employees_c.head()

,city,company_founded_year,company_linkedin_uid,company_name,company_size,country,email,full_name,functional_level,headline,industry,job_title,linkedin,seniority_level,state
0,NaN,2001,2845844,Dahua Technology Co. Ltd,23000,Croatia,damir.klarin@dahuatech.com,Damir Klarin,sales,NaN,Information Technology & Services,Business Development Manager,http://www.linkedin.com/in/damir-klarin-8496283,manager,NaN
1,Dubai,2001,2845844,Dahua Technology Co. Ltd,23000,United Arab Emirates,NaN,Ali Kloub,sales,Regional Director (mena)\r\nbusiness Growth | ...,Information Technology & Services,Regional Sales Director (mena),http://www.linkedin.com/in/ali-kloub-b15b2657,director,Dubai
2,Jeddah,2001,2845844,Dahua Technology Co. Ltd,23000,Saudi Arabia,yahya.asfari@dahuatech.com,Yahya Asfari,engineering,Director Of Business Development | Ai Surveill...,Information Technology & Services,Director Of Business Development | Ai Surveill...,http://www.linkedin.com/in/yahya1asfari,director,Makkah Province
3,Bogota,2001,2845844,Dahua Technology Co. Ltd,23000,Colombia,ruben.trujillo@dahuatech.com,Jose Ruben Trujillo,sales,Business Development Director,Information Technology & Services,Business Development Director,http://www.linkedin.com/in/jruben-trujillo,director,Bogota
4,Beirut,2001,2845844,Dahua Technology Co. Ltd,23000,Lebanon,NaN,Rodolphe Kassouf,sales,Director Of Business Development,Information Technology & Services,Director Of Business Development,http://www.linkedin.com/in/rodolphe-kassouf,director,Beirut Governorate


In [14]:
df_employees_c.tail()

,city,company_founded_year,company_linkedin_uid,company_name,company_size,country,email,full_name,functional_level,headline,industry,job_title,linkedin,seniority_level,state
95,Nairobi,2001,2845844,Dahua Technology Co. Ltd,23000,Kenya,zephania.mwando@dahuatech.com,Zephania Mwando,NaN,Solution Manager,Information Technology & Services,Solution Manager,http://www.linkedin.com/in/mwandotheboss,manager,Nairobi
96,NaN,2001,2845844,Dahua Technology Co. Ltd,23000,Slovakia,marek.vavra@dahuatech.com,Marek Vavra,sales,--,Information Technology & Services,Country Sales Manager Cz&sk,http://www.linkedin.com/in/marek-vavra-69b4a6133,manager,NaN
97,Belgrade,2001,2845844,Dahua Technology Co. Ltd,23000,Serbia,thierry.sun@dahuatech.com,Thierry Sun,operations,Distribution Manager At Dahua Technology Co. Ltd,Information Technology & Services,Distribution Manager,http://www.linkedin.com/in/thierry-sun-3b3a0225b,manager,NaN
98,Zoetermeer,2001,2845844,Dahua Technology Co. Ltd,23000,Netherlands,ruan_weijie@dahuatech.com,Ruan Weijie,marketing,Marketing Manager Western Europe At Dahua Tech...,Information Technology & Services,Marketing Manager - Western Europe,http://www.linkedin.com/in/wjruan,manager,South Holland
99,Riyadh,2001,2845844,Dahua Technology Co. Ltd,23000,Saudi Arabia,tariq@securityworld.com.sa,Tariq Qashou,engineering,NaN,Information Technology & Services,Senior Solutions Engineer,http://www.linkedin.com/in/tariq-qashou-a1760a54,senior,Riyadh Province


In [15]:
df_employees_c[df_employees_c['full_name'] == 'Wu Jian']

,city,company_founded_year,company_linkedin_uid,company_name,company_size,country,email,full_name,functional_level,headline,industry,job_title,linkedin,seniority_level,state


In [16]:
df_employees_c['full_name'].nunique()

100

In [17]:
df_employees_c['seniority_level'].value_counts()

seniority_level
manager     58
director    22
senior       6
head         6
vp           4
partner      3
c_suite      1
Name: count, dtype: int64

In [18]:
df_crunchbase_employees = pd.read_csv(r"C:\Users\LENOVO\Documents\GitHub\Enrichment_database\df_crunchbase_employees.csv", sep = ";")
df_crunchbase_employees.head()


,company_id,name,title,website
0,dahua-technology,Zhang Xingming,Executive President,https://www.dahuatech.com
1,dahua-technology,Wu Jian,Senior Vice President,https://www.dahuatech.com
2,dahua-technology,Jiang Xiaolai,Senior Vice President,https://www.dahuatech.com
3,dahua-technology,Zhu Jiantang,Senior Vice President,https://www.dahuatech.com
4,dahua-technology,Song Ke,Vice President,https://www.dahuatech.com
